# imports

In [1]:
import wandb

wandb.login(key="c4808bcfac97e4f6678a2e24adcc3b6c37427e2e")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [2]:
!pip -q install evaluate

In [ ]:
from datasets import load_dataset, Dataset

import datasets
import torch

import numpy as np
import pandas as pd
from tqdm.auto import tqdm


from transformers import TrainingArguments, Trainer
import evaluate


from accelerate import Accelerator
from accelerate.utils import set_seed 

from transformers import GenerationConfig
from torch.optim import AdamW
# from transformers import get_scheduler
import torch.optim.lr_scheduler as lr_scheduler
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding

# load data, toeknizer and model

In [4]:
train_df = pd.read_csv('/kaggle/input/nlp-with-disaster-tweets/train_preprocessed.csv')
test_df = pd.read_csv('/kaggle/input/nlp-with-disaster-tweets/test_preprocessed.csv')

In [5]:
train_df.head()

,keyword,location,text,target
0,neutral,us,Our Deeds Reason earthquake May ALLAH Forgive us,1
1,fire,canada,Forest fire near La Ronge Sask Canada,1
2,evacuation,nowhere,All residents asked shelter place notified off...,1
3,evacuation,california,people receive wildfires evacuation orders Cal...,1
4,smoke,nowhere,Just got sent photo Ruby Alaska smoke wildfire...,1


In [6]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [7]:
train_ds = Dataset.from_pandas(train_df)
test_ds = Dataset.from_pandas(test_df)

In [8]:
train_ds

Dataset({
    features: ['keyword', 'location', 'text', 'target'],
    num_rows: 7613
})

In [9]:
checkpoint = "facebook/bart-large-cnn"  # accuracy .824988

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [11]:
id2label = {0: "Not Disaster", 1: "Disaster"}
label2id = {"Not Disaster": 0, "Disaster": 1}

In [12]:
base_model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2, id2label=id2label, label2id=label2id)

Some weights of BartForSequenceClassification were not initialized from the model checkpoint at facebook/bart-large-cnn and are newly initialized: ['classification_head.dense.bias', 'classification_head.out_proj.weight', 'classification_head.dense.weight', 'classification_head.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
base_model

BartForSequenceClassification(
  (model): BartModel(
    (shared): Embedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): Layer

# Preprocessing

In [ ]:
def tokenize_example(example):
    return tokenizer(example["text"], example["keyword"], truncation=True)

tokenized_train_data = train_ds.map(tokenize_example, batched=True)
tokenized_test_data = test_ds.map(tokenize_example, batched=True, remove_columns = test_ds.column_names)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [15]:
tokenized_train_data

Dataset({
    features: ['keyword', 'location', 'text', 'target', 'input_ids', 'attention_mask'],
    num_rows: 7613
})

In [16]:
tokenized_train_data = tokenized_train_data.remove_columns(["keyword", "location", "text"])
tokenized_train_data = tokenized_train_data.rename_column("target", "labels")
tokenized_train_data.set_format("torch")
tokenized_test_data.set_format("torch")
tokenized_train_data.features, tokenized_test_data.features

({'labels': Value(dtype='int64', id=None),
  'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
  'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)},
 {'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
  'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)})

In [17]:
train_val_split = tokenized_train_data.train_test_split(0.2, shuffle=True)

In [18]:
train_val_split

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 6090
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 1523
    })
})

In [24]:
BATCH_SIZE = 64

from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    train_val_split["train"], shuffle=True, batch_size=BATCH_SIZE, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    train_val_split["test"], batch_size=BATCH_SIZE, collate_fn=data_collator
)

test_dataloader = DataLoader(
    tokenized_test_data, batch_size=BATCH_SIZE, collate_fn=data_collator
)

In [20]:
# for batch in train_dataloader:
#     break
# {k: v.shape for k, v in batch.items()}

In [ ]:
accuracy = evaluate.load("accuracy")



def compute_metrics(eval_pred):

    predictions, labels = eval_pred

    # Use np.argmax to get the predicted class indices from logits
    predictions = np.argmax(predictions[0], axis=1)
    

    # If the labels are one-hot encoded, convert them to class indices
    if len(labels.shape) > 1:
        labels = np.argmax(labels, axis=-1)

    return accuracy.compute(predictions=predictions, references=labels)

In [22]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")

training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=4,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=base_model,
    args=training_args,
    train_dataset=train_val_split["train"],
    eval_dataset=train_val_split["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: Currently logged in as: skt27182. Use `wandb login --relogin` to force relogin


You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.390707,0.836507
2,0.424900,0.419793,0.835850
3,0.287200,0.559149,0.807617
4,0.188000,0.702040,0.820092


TrainOutput(global_step=1524, training_loss=0.2971298661757642, metrics={'train_runtime': 603.7477, 'train_samples_per_second': 40.348, 'train_steps_per_second': 2.524, 'total_flos': 1591338944595552.0, 'train_loss': 0.2971298661757642, 'epoch': 4.0})

- We can clearly see that after initial 2 epochs it started over-fitting. As we have given the argument `load_best_model_at_end=True` so it will load the best model at end.

In [25]:
import evaluate

metric = evaluate.load( "accuracy")

base_model.eval()
for batch in tqdm(eval_dataloader):
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = base_model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

  0%|          | 0/24 [00:00<?, ?it/s]

{'accuracy': 0.8365068942875903}

In [26]:
# optimizer = AdamW(base_model.parameters(), lr=2e-05)

# num_epochs = 10
# num_training_steps = num_epochs * len(train_dataloader)

# lr_scheduler = lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0, total_iters=num_training_steps)

# print(num_training_steps)

# base_model.to(device)


# from tqdm.auto import tqdm

# progress_bar = tqdm(range(num_training_steps))

# base_model.train()
# for epoch in range(num_epochs):
#     for batch in train_dataloader:
#         batch = {k: v.to(device) for k, v in batch.items()}
#         outputs = base_model(**batch)
#         loss = outputs.loss
#         loss.backward()

#         optimizer.step()
#         lr_scheduler.step()
#         optimizer.zero_grad()
#         progress_bar.set_postfix(loss=loss.item())
#         progress_bar.update(1)

# Final Test

In [27]:
import numpy as np

pred = []

base_model.eval()
for batch in test_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = base_model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    pred.append(predictions.reshape(-1).cpu().numpy())

# Convert the list of arrays into a 1D NumPy array
pred = np.concatenate(pred)

In [28]:
y_pred = pred.reshape(-1,1)

In [29]:
ids = pd.read_csv('/kaggle/input/nlp-with-disaster-tweets/test_id.csv')['id']

# merge ids and predictions
submission = pd.DataFrame(np.concatenate((ids.values.reshape(-1, 1), y_pred.reshape(-1,1)), axis=1), columns=['id', 'target'])

# make id int   
submission['id'] = submission['id'].astype('int')

# make target 0 and 1
submission['target'] = submission['target'].apply(lambda x: 0 if x < 0.5 else 1)

In [33]:
submission.head()

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1


In [32]:
# Save submission
submission.to_csv('submission.csv', index=False)